## Agents Demystified - part 2

## Introduction

In this tutorial we'll build on the results from [part 1](agents_pt1.ipynb) and explore letting the agent write and execute code, and interact with data. In order to keep the notebook uncluttered, the code from part 1 is contained in a separate [file](agents2.py) that we'll import and extend.

So, without further ado, let's get started!

In [ ]:
!pip -q install ollama tavily

In [ ]:
import os

from data.agents2 import Agent
from data.agents2 import (date, calculator, web_search)

In [ ]:
# Host and model definitions
OLLAMA_HOST = 'http://10.129.20.4:9090'
OLLAMA_MODEL = 'gemma3:27b-it-qat'

os.environ['TAVILY_API_KEY'] = "" # <-- paste your key between the quotes before running this cell

## Baseline test

Just to make sure everything works as expected (i.e. where we left off in the previuos part):

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[date, web_search, calculator])
print(agent.task("what is the sum of the today's noon temperature in Paris and Berlin? Answer in centigrades. Is the answer reasonable given today's date?"))

In [ ]:
print(agent.message_history())

## Agent 007 with licence to code

As we saw before, asking our agent for the time without providing a helper tool results in an hallucination:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[])
print(agent.task("What time is it?"))

This time, we'll take a more roundabout way to solve the problem: Let the agent code its own helper!

For this we need a generic tool to execute a python script<sup>1</sup> that the agent can call with the source code as argument.

---
<p><small>1. Allowing agents to execute code is obviously dangerous, and in a real application this tool would have to be sandboxed and locke down but as that is outside the scope of this tutorial no restrictions apply. You have been warned, be careful with what you ask the agent to do.</small></p>

---

In [ ]:
def execute_script(script: str) -> str:
    """
    Excecute python code and return the result as a string.
    You may import any python module, e.g. datetime or pandas
    If the script produce a figure, write it to a PNG file in the current working directory and return its name as a string using the format '## Figure: [name] ##' so it is visible to the user,

    Args:
        script (str): The python script to evaluate

    Returns:
        str: the result of running the script or an error message in case of failure

    """
    import subprocess
    import os
    import re

    from IPython.display import Image, display_png
    
    script_filename = "temp_script.py"
    output = ""
    
    with open(script_filename, "w") as script_file:
        script_file.write(script)
    try:
        result = subprocess.run(
            ["python", script_filename],
            capture_output=True,
            text=True,
            check=True,
        )
        output = result.stdout  
        # Uncomment next line to get debbuging output
        # print("Script output:", output) 
    except subprocess.CalledProcessError as e:
        print("Script execution failed:", e.stderr)  
    finally:
        pass
        # Uncomment to clean up temp file (script)
        # if os.path.exists(script_filename):
        #    os.remove(script_filename)

    # Check if a figure was produced and 
    RE_FIG = re.compile(r'## Figure:\s*(\S+)')
    fig_match = RE_FIG.match(output)
    if fig_match:
        fig_path = fig_match.group(1)
        display_png(Image(filename=fig_path))

    return output

This is a vanilla use of python's `subprocess` module to invoke python on a script (`temp_script.py`) containing the code written by the agent. If a an image was produced by the script (as per the docstring) it is rendered with the output.

In [ ]:
# Check for syntax errors in execute_script()
execute_script("print('Hello')")

Now we can ask for the time again, this time providing `execute_script` as the only tool:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
print(agent.task("What time is it?"))

In [ ]:
print(agent.message_history())

That's it!

Without any further changes, we can now give the agent complex queries, involving data analysis and visualiztion:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
print(agent.task("You have permission to load the data file 'titanic.csv' in your default working directory. The file contains facts about the fate of the passengers abord the Titanic. Give me a verbal summary of the main conclusions that can be drawn from those facts."))

In [ ]:
# print(agent.message_history())

If we don't instantiate a new agent, we can continue to ask questions about the data …

In [ ]:
# same agent, continued...
print(agent.task("How many passengers were there, and how many survived?"))

… and we can ask for visualizations of data:

In [ ]:
# same agent, continued...
print(agent.task("Display a diagram showing survival versus fare"))

In [ ]:
# print(agent.message_history())

In [ ]:
# same agent, continued...
print(agent.task("Not quite what I wanted. Can you show me a diagram of likelihood of survival (in percent) versus fare? Bin the fare into suitable ranges."))

In [ ]:
print(agent.message_history())